In [15]:
from pytorch3d.renderer import look_at_view_transform
from pytorch3d.io import load_obj
import torch
from pytorch3d.renderer import FoVPerspectiveCameras
import pytorch3d

vertices, faces, _ = load_obj("data/cow.obj")
vertices = vertices.unsqueeze(0)
faces = faces.verts_idx.unsqueeze(0)

texture_rgb = torch.ones_like(vertices) # N X 3
texture_rgb = texture_rgb * torch.tensor([0.7, 0.7, 1])
textures = pytorch3d.renderer.TexturesVertex(texture_rgb) #

meshes = pytorch3d.structures.Meshes(
    verts=vertices, # batched tensor or a list of tensors
    faces=faces,
    textures=textures,
)
meshes

In [16]:
# Define the number of cameras for each direction
num_azimuth = 18  # horizontal rotation (every 10 degrees)
num_elevation = 18  # vertical rotation (every 10 degrees)
num_cameras = num_azimuth * num_elevation

# Create evenly spaced angles for both azimuth and elevation
azim = torch.linspace(0, 360, num_azimuth)
elev = torch.linspace(-80, 80, num_elevation)  # Avoiding exact poles (-90/90) to prevent numerical issues

# Create a grid of all angle combinations
azim_grid, elev_grid = torch.meshgrid(azim, elev, indexing='ij')
azim_all = azim_grid.flatten()
elev_all = elev_grid.flatten()

# Define the distance of the cameras from the object
R, T = look_at_view_transform(
    dist=3,
    elev=elev_all,
    azim=azim_all
)

# Create a batch of cameras
cameras = FoVPerspectiveCameras(device='cpu', R=R, T=T)

In [17]:
from pytorch3d.vis.plotly_vis import plot_scene
plot_scene({
    "360-degree Renders": {
        "Mesh": meshes,
        "Cameras": cameras,
    }
})

In [18]:
from pytorch3d.renderer import (
    RasterizationSettings,
    MeshRenderer,
    MeshRasterizer,
    SoftPhongShader,
    PointLights,
)

from starter.utils import get_mesh_renderer

# Set the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Move the mesh to the device
meshes = meshes.to(device)

# Create a renderer
raster_settings = RasterizationSettings(
    image_size=512,
    blur_radius=0.0,
    faces_per_pixel=1,
)

lights = PointLights(device=device, location=[[0.0, 0.0, -3.0]])

renderer = get_mesh_renderer(lights=lights, image_size=512, device=device)
# renderer = MeshRenderer(
#     rasterizer=MeshRasterizer(
#         cameras=cameras,
#         raster_settings=raster_settings
#     ),
#     shader=SoftPhongShader(
#         device=device,
#         cameras=cameras,
#         lights=lights
#     )
# )

# Render the images
images = renderer(meshes.extend(num_cameras), cameras=cameras, lights=lights)

In [19]:
from PIL import Image
import numpy as np
images = images.cpu().numpy()[:, :, :, :3] # [1, H, W, rgb]
images = [(image*255).astype(np.uint8) for image in images]

In [20]:
import imageio
duration = 1000 // 15  # Convert FPS (frames per second) to duration (ms per frame)
imageio.mimsave('play/loop-circle-cameras.gif', images, duration=duration, loop=0)